# Procesamiento en PySpark

Configuración del context para usar PySpark

In [4]:
import sys
print(sys.executable)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/usr/bin/python3

In [11]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
%%configure -f
{ "conf":{
"spark.pyspark.python": "python3",
"spark.pyspark.virtualenv.enabled": "true",
"spark.pyspark.virtualenv.type":"native",
"spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
}}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1725594577268_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1725594577268_0002,pyspark,idle,Link,Link,None,✔


In [16]:
sc.list_packages()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Package                    Version
-------------------------- ------------
aws-cfn-bootstrap          2.0
beautifulsoup4             4.9.3
boto                       2.49.0
click                      8.1.7
docutils                   0.14
jmespath                   1.0.1
joblib                     1.3.2
lockfile                   0.11.0
lxml                       4.9.3
mysqlclient                1.4.2
nltk                       3.8.1
nose                       1.3.4
numpy                      1.20.0
pip                        20.2.2
py-dateutil                2.2
pystache                   0.5.4
python-daemon              2.2.3
python37-sagemaker-pyspark 1.4.2
pytz                       2023.3.post1
PyYAML                     5.4.1
regex                      2021.11.10
setuptools                 28.8.0
simplejson                 3.2.0
six                        1.13.0
tqdm                       4.66.1
wheel                      0.29.0
windmill                   1.6


Conexión con la base de datos:

In [23]:
df = spark.sql("show databases")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+
| namespace|
+----------+
|   default|
|    labsdb|
|usernamedb|
+----------+

Contamos con 3 bases de datos y la que usaremos sera labsdb.

# Consultas de las tablas

# Tabla Atm_con: Hacemos un select para traernos todos los campos y conocer la información con la que contamos

In [2]:
df2 = spark.sql("select *, SUBSTRING(date, 1, 4) AS year from labsdb.atm_con where objectid is not null")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df2.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+----+----+--------------------+---------------+--------------------+--------+--------------------+--------------------+-------+------+----+
|objectid|country|iso2|iso3|           indicator|           unit|              source|cts_code|            cts_name| cts_full_descriptor|   date| value|year|
+--------+-------+----+----+--------------------+---------------+--------------------+--------+--------------------+--------------------+-------+------+----+
|       1|  World|    | WLD|MonthlyAtmospheri...|PartsPerMillion|Dr.PieterTansNati...|    ECCA|AtmosphericCarbon...|EnvironmentClimat...|1958M03|315.71|1958|
|       2|  World|    | WLD|MonthlyAtmospheri...|PartsPerMillion|Dr.PieterTansNati...|    ECCA|AtmosphericCarbon...|EnvironmentClimat...|1958M04|317.45|1958|
|       3|  World|    | WLD|MonthlyAtmospheri...|PartsPerMillion|Dr.PieterTansNati...|    ECCA|AtmosphericCarbon...|EnvironmentClimat...|1958M05|317.51|1958|
|       4|  World|    | WLD|MonthlyAtmospheri...|Par

Conteo de palabras para conocer cuales son las palabras mas repetidas en la tabla de atmosfera (atm_con):

In [1]:
files_rdd = sc.textFile("s3://trustedtrabajo1/trusted/atm_con.txt")
wc_unsort = files_rdd.flatMap(lambda line: line.split()).map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
wc = wc_unsort.sortBy(lambda a: -a[1])
for tupla in wc.take(10):
        print(tupla)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1725738867665_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

('Carbon', 7910)
('Atmospheric', 7910)
('Dioxide', 7910)
('and', 4746)
('Concentrations', 3961)
('Climate', 3164)
('Dr.', 3164)
('on', 2367)
('World', 1582)
('National', 1582)

In [36]:
from pyspark.sql import SparkSession

# spark local:
spark = SparkSession.builder.master("local[*]").getOrCreate()

# spark conectado con Amazon AWS:

spark = SparkSession.builder.appName("data_processing").master("local[*]").config("spark.driver.memory","16G").config("spark.driver.maxResultSize", "0").config("spark.kryoserializer.buffer.max", "2000M").config("spark.jars.packages", "com.amazonaws:aws-java-sdk:1.11.1034,org.apache.hadoop:hadoop-aws:3.3.4").config('fs.s3a.access.key', "AWS_ACCESS_KEY").config('fs.s3a.secret.key', "AWS_SECRET_KEY").config('fs.s3a.session.token',"AWS_SESSION_TOKEN").config('fs.s3a.path.style.access', 'true').config('fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem').config('fs.s3a.endpoint', 's3.amazonaws.com').getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Aca extraemos las datablas directamente desde el bucket de S3:

In [4]:
df2=spark.read.csv('s3://rawtrabajo1/raw/ATM_CON/atm_con.csv',inferSchema=True,header=True, sep = ';')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
print(df2.columns) #Mostramos los nombres de las columnas que contiene la tabla
print(len(df2.columns)) #Numero de columnas
print(df2.count()) #Cantidad de datos que contiene el dataframe
print((df2.count(),len(df.columns))) #Tamaño del dataframe (cantidad de columnas y filas)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'df' is not defined
Traceback (most recent call last):
NameError: name 'df' is not defined



Mostramos el esquema de como esta creada la tabla: las columnas que contiene, el tipo de dato de cada columna y si cuenta con nulos o no:

In [109]:
df2.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- objectid: long (nullable = true)
 |-- country: string (nullable = true)
 |-- iso2: string (nullable = true)
 |-- iso3: string (nullable = true)
 |-- indicator: string (nullable = true)
 |-- unit: string (nullable = true)
 |-- source: string (nullable = true)
 |-- cts_code: string (nullable = true)
 |-- cts_name: string (nullable = true)
 |-- cts_full_descriptor: string (nullable = true)
 |-- date: string (nullable = true)
 |-- value: string (nullable = true)
 |-- year: string (nullable = true)

df.printSchema() 

In [111]:
df2.show(5)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+----+----+--------------------+---------------+--------------------+--------+--------------------+--------------------+-------+------+----+
|objectid|country|iso2|iso3|           indicator|           unit|              source|cts_code|            cts_name| cts_full_descriptor|   date| value|year|
+--------+-------+----+----+--------------------+---------------+--------------------+--------+--------------------+--------------------+-------+------+----+
|       1|  World|    | WLD|MonthlyAtmospheri...|PartsPerMillion|Dr.PieterTansNati...|    ECCA|AtmosphericCarbon...|EnvironmentClimat...|1958M03|315.71|1958|
|       2|  World|    | WLD|MonthlyAtmospheri...|PartsPerMillion|Dr.PieterTansNati...|    ECCA|AtmosphericCarbon...|EnvironmentClimat...|1958M04|317.45|1958|
|       3|  World|    | WLD|MonthlyAtmospheri...|PartsPerMillion|Dr.PieterTansNati...|    ECCA|AtmosphericCarbon...|EnvironmentClimat...|1958M05|317.51|1958|
|       4|  World|    | WLD|MonthlyAtmospheri...|Par

Hacemos un select para conocer los paises, el indicador, la fecha y el valor de las concentraciones de carbono en la atmosfera:

In [112]:
df2.select('Country','Indicator', 'year' ,'value').show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+----+------+
|Country|           Indicator|year| value|
+-------+--------------------+----+------+
|  World|MonthlyAtmospheri...|1958|315.71|
|  World|MonthlyAtmospheri...|1958|317.45|
|  World|MonthlyAtmospheri...|1958|317.51|
|  World|MonthlyAtmospheri...|1958|317.27|
|  World|MonthlyAtmospheri...|1958|315.87|
+-------+--------------------+----+------+
only showing top 5 rows

Sacamos estadisticos para analizar el comportamiento de cada variable como la cantidad de datos, el promedio, valores minimos y maximos, como value es la unica variable que tenemos numerica, calculamos los estadisticos para esta:

In [85]:
df2.describe('value').show() #estadisticos para cada variable

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+
|summary|             value|
+-------+------------------+
|  count|              1582|
|   mean|181.23111251580275|
| stddev|180.88644331662368|
|    min|             -0.11|
|    max|            426.91|
+-------+------------------+

Hacemos procesamiento de las variables:

In [86]:
from pyspark.sql.types import StringType,DoubleType,IntegerType

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [91]:
df2.withColumn('value_double',df2['value'].cast(DoubleType())).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+----+----+--------------------+---------------+--------------------+--------+--------------------+--------------------+-------+------+------------+
|ObjectId|Country|ISO2|ISO3|           Indicator|           Unit|              Source|CTS_Code|            CTS_Name| CTS_Full_Descriptor|   Date| Value|value_double|
+--------+-------+----+----+--------------------+---------------+--------------------+--------+--------------------+--------------------+-------+------+------------+
|       1|  World|null| WLD|MonthlyAtmospheri...|PartsPerMillion|Dr.PieterTansNati...|    ECCA|AtmosphericCarbon...|EnvironmentClimat...|1958M03|315.71|      315.71|
|       2|  World|null| WLD|MonthlyAtmospheri...|PartsPerMillion|Dr.PieterTansNati...|    ECCA|AtmosphericCarbon...|EnvironmentClimat...|1958M04|317.45|      317.45|
|       3|  World|null| WLD|MonthlyAtmospheri...|PartsPerMillion|Dr.PieterTansNati...|    ECCA|AtmosphericCarbon...|EnvironmentClimat...|1958M05|317.51|      317.51|
|   

In [95]:
df2.filter(df2['unit']=='Percent').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+----+----+--------------------+-------+--------------------+--------+--------------------+--------------------+-------+-----+
|objectid|country|iso2|iso3|           indicator|   unit|              source|cts_code|            cts_name| cts_full_descriptor|   date|value|
+--------+-------+----+----+--------------------+-------+--------------------+--------+--------------------+--------------------+-------+-----+
|      14|  World|    | WLD|MonthlyAtmospheri...|Percent|Dr.PieterTansNati...|    ECCA|AtmosphericCarbon...|EnvironmentClimat...|1959M03|  0.3|
|      16|  World|    | WLD|MonthlyAtmospheri...|Percent|Dr.PieterTansNati...|    ECCA|AtmosphericCarbon...|EnvironmentClimat...|1959M04| 0.09|
|      18|  World|    | WLD|MonthlyAtmospheri...|Percent|Dr.PieterTansNati...|    ECCA|AtmosphericCarbon...|EnvironmentClimat...|1959M05| 0.25|
|      20|  World|    | WLD|MonthlyAtmospheri...|Percent|Dr.PieterTansNati...|    ECCA|AtmosphericCarbon...|EnvironmentClimat...|1959M06

In [6]:
df2.filter(df['unit']=='Percent').select('Country','indicator','unit', 'year','value').show() #porcentaje 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'df' is not defined
Traceback (most recent call last):
NameError: name 'df' is not defined



In [8]:
df2.filter(df2['unit']=='Percent').filter(df2['value'] > 0.5 ).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+----+----+--------------------+-------+--------------------+--------+--------------------+--------------------+-------+-----+
|ObjectId|Country|ISO2|ISO3|           Indicator|   Unit|              Source|CTS_Code|            CTS_Name| CTS_Full_Descriptor|   Date|Value|
+--------+-------+----+----+--------------------+-------+--------------------+--------+--------------------+--------------------+-------+-----+
|      42|  World|null| WLD|MonthlyAtmospheri...|Percent|Dr.PieterTansNati...|    ECCA|AtmosphericCarbon...|EnvironmentClimat...|1960M05| 0.55|
|      46|  World|null| WLD|MonthlyAtmospheri...|Percent|Dr.PieterTansNati...|    ECCA|AtmosphericCarbon...|EnvironmentClimat...|1960M07| 0.52|
|     186|  World|null| WLD|MonthlyAtmospheri...|Percent|Dr.PieterTansNati...|    ECCA|AtmosphericCarbon...|EnvironmentClimat...|1966M05| 0.59|
|     188|  World|null| WLD|MonthlyAtmospheri...|Percent|Dr.PieterTansNati...|    ECCA|AtmosphericCarbon...|EnvironmentClimat...|1966M06

In [98]:
df2.select('indicator').distinct().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|           indicator|
+--------------------+
|MonthlyAtmospheri...|
|MonthlyAtmospheri...|
+--------------------+

In [99]:
df2.select('indicator').distinct().count()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2

In [100]:
df2.groupBy('indicator').count().show(5,False)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------------------------------------------------------+-----+
|indicator                                                              |count|
+-----------------------------------------------------------------------+-----+
|MonthlyAtmosphericCarbonDioxideConcentrationsYearonYearPercentageChange|785  |
|MonthlyAtmosphericCarbonDioxideConcentrations                          |797  |
+-----------------------------------------------------------------------+-----+

In [101]:
df2.groupBy('indicator').count().orderBy('count',ascending=False).show(5,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------------------------------------------------------+-----+
|indicator                                                              |count|
+-----------------------------------------------------------------------+-----+
|MonthlyAtmosphericCarbonDioxideConcentrations                          |797  |
|MonthlyAtmosphericCarbonDioxideConcentrationsYearonYearPercentageChange|785  |
+-----------------------------------------------------------------------+-----+

In [116]:
df2.groupBy('year').sum().show(5,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-------------+
|year|sum(objectid)|
+----+-------------+
|1962|2220         |
|1964|3372         |
|1965|3948         |
|1966|4524         |
|1963|2796         |
+----+-------------+
only showing top 5 rows

In [119]:
print(df2.groupBy('year').max().show(5,False))
print(df2.groupBy('year').min().show(5,False))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-------------+
|year|max(objectid)|
+----+-------------+
|1962|104          |
|1964|152          |
|1965|176          |
|1966|200          |
|1963|128          |
+----+-------------+
only showing top 5 rows

None
+----+-------------+
|year|min(objectid)|
+----+-------------+
|1962|81           |
|1964|129          |
|1965|153          |
|1966|177          |
|1963|105          |
+----+-------------+
only showing top 5 rows

None

In [120]:
df2.groupBy('year').agg({'value':'sum'}).show(5,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+------------------+
|year|sum(value)        |
+----+------------------+
|1962|3824.5100000000007|
|1964|3837.79           |
|1965|3841.999999999999 |
|1966|3861.39           |
|1963|3829.93           |
+----+------------------+
only showing top 5 rows

In [11]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, FloatType


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [122]:
def cant_carbono(value):
    try:
        value = float(value) 
        if value < 412:
            return 'Bajo carbono'
        else:
            return 'Alto carbono'
    except ValueError:
        return 'Invalid value'  

cant_udf = udf(cant_carbono, StringType())
df2 = df2.withColumn('cant_carbono', cant_udf(df2['value']))
df2.show(10, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+----+----+---------------------------------------------+---------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+--------------------------------------+-------------------------------------------------------------------------------+-------+------+----+------------+
|objectid|country|iso2|iso3|indicator                                    |unit           |source                                                                                                                                                                                                                                                         |cts_code|cts_name                              |cts_full_descriptor                                                            |date   |value |yea

In [53]:
from pyspark.sql.functions import pandas_udf, PandasUDFType


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [123]:
def remaining_yrs(date):
    yrs_left=100-year

    return yrs_left

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [125]:
# eliminar duplicados
print(df2.count())
df2=df2.dropDuplicates()
print(df2.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1582
1582

In [129]:
#drop column of dataframe
df2_new=df2.drop('ISO3')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [131]:
df2_new.show(10)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+----+--------------------+---------------+--------------------+--------+--------------------+--------------------+-------+------+----+------------+
|objectid|country|iso2|           indicator|           unit|              source|cts_code|            cts_name| cts_full_descriptor|   date| value|year|cant_carbono|
+--------+-------+----+--------------------+---------------+--------------------+--------+--------------------+--------------------+-------+------+----+------------+
|      35|  World|    |MonthlyAtmospheri...|PartsPerMillion|Dr.PieterTansNati...|    ECCA|AtmosphericCarbon...|EnvironmentClimat...|1960M02|316.98|1960|Bajo carbono|
|      77|  World|    |MonthlyAtmospheri...|PartsPerMillion|Dr.PieterTansNati...|    ECCA|AtmosphericCarbon...|EnvironmentClimat...|1961M11| 316.1|1961|Bajo carbono|
|     162|  World|    |MonthlyAtmospheri...|        Percent|Dr.PieterTansNati...|    ECCA|AtmosphericCarbon...|EnvironmentClimat...|1965M05| -0.02|1965|Bajo carbono|
|   

In [12]:
df2_sin_null = df2.withColumn("value", df2["value"].cast("float"))

#Agregar la clasificación por medio de lambda
cant_carbono_udf = udf(lambda value: "Bajo carbono" if value < 412 else "Alto carbono", StringType())

df2_sin_null.withColumn("cant_carbono2", cant_carbono_udf(df2_sin_null.value)).show(10,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+----+----+---------------------------------------------+---------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+--------------------------------------+-------------------------------------------------------------------------------+-------+------+-------------+
|ObjectId|Country|ISO2|ISO3|Indicator                                    |Unit           |Source                                                                                                                                                                                                                                                                                         |CTS_Code|CTS_Name                              |CTS_Full_Descriptor                   

In [13]:
 write_uri='s3://trustedtrabajo1/trusted/spark_atm'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
df2_sin_null.coalesce(1).write.format("csv").option("header","true").save(write_uri)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Tabla Temp_change: Hacemos un select para traernos todos los campos y conocer la información con la que contamos

In [2]:
df = spark.sql("select * from labsdb.temp_change")
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+----+----+--------------------+--------------+--------------------+--------+--------------------+--------------------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-------+--------+-------+-------+--------+--------+-------+-------+-------+-------+--------+-------+-------+--------+-------+-------+-------+-------+-------+--------+-------+-------+
|objectid|             country|iso2|iso3|           indicator|          unit|              source|cts_code|            cts_name| cts_full_descriptor|   f1961|   f1962|   f1963|   f1964|   f1965|   f1966|   f1967|   f1968|   f1969|   f1970|   f1971|   f1972|   f1973|   f1974|   f1975|   f1976|   f197

In [3]:
files_rdd = sc.textFile("s3://trustedtrabajo1/trusted/temp_change.txt")
wc_unsort = files_rdd.flatMap(lambda line: line.split()).map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
wc = wc_unsort.sortBy(lambda a: -a[1])
for tupla in wc.take(10):
        print(tupla)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

('Climate', 944)
('Temperature', 944)
('and', 482)
('the', 477)
('Surface', 472)
('Change', 472)
('to', 472)
('Change,', 472)
('of', 282)
('United', 241)

In [7]:
df=spark.read.csv('s3://rawtrabajo1/raw/TEMP_CHANGE/temp_change.csv',inferSchema=True,header=True, sep = ';')
print(df.columns) #Mostramos los nombres de las columnas que contiene la tabla
print(len(df.columns)) #Numero de columnas
print(df.count()) #Cantidad de datos que contiene el dataframe
print((df.count(),len(df.columns))) #Tamaño del dataframe (cantidad de columnas y filas)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['ObjectId', 'Country', 'ISO2', 'ISO3', 'Indicator', 'Unit', 'Source', 'CTS_Code', 'CTS_Name', 'CTS_Full_Descriptor', 'F1961', 'F1962', 'F1963', 'F1964', 'F1965', 'F1966', 'F1967', 'F1968', 'F1969', 'F1970', 'F1971', 'F1972', 'F1973', 'F1974', 'F1975', 'F1976', 'F1977', 'F1978', 'F1979', 'F1980', 'F1981', 'F1982', 'F1983', 'F1984', 'F1985', 'F1986', 'F1987', 'F1988', 'F1989', 'F1990', 'F1991', 'F1992', 'F1993', 'F1994', 'F1995', 'F1996', 'F1997', 'F1998', 'F1999', 'F2000', 'F2001', 'F2002', 'F2003', 'F2004', 'F2005', 'F2006', 'F2007', 'F2008', 'F2009', 'F2010', 'F2011', 'F2012', 'F2013', 'F2014', 'F2015', 'F2016', 'F2017', 'F2018', 'F2019', 'F2020', 'F2021', 'F2022', 'F2023']
73
236
(236, 73)

In [8]:
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ObjectId: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- ISO2: string (nullable = true)
 |-- ISO3: string (nullable = true)
 |-- Indicator: string (nullable = true)
 |-- Unit: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- CTS_Code: string (nullable = true)
 |-- CTS_Name: string (nullable = true)
 |-- CTS_Full_Descriptor: string (nullable = true)
 |-- F1961: double (nullable = true)
 |-- F1962: double (nullable = true)
 |-- F1963: double (nullable = true)
 |-- F1964: double (nullable = true)
 |-- F1965: double (nullable = true)
 |-- F1966: double (nullable = true)
 |-- F1967: double (nullable = true)
 |-- F1968: double (nullable = true)
 |-- F1969: double (nullable = true)
 |-- F1970: double (nullable = true)
 |-- F1971: double (nullable = true)
 |-- F1972: double (nullable = true)
 |-- F1973: double (nullable = true)
 |-- F1974: double (nullable = true)
 |-- F1975: double (nullable = true)
 |-- F1976: double (nullable = true)
 |-- 

In [25]:
df.head(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(ObjectId=1, Country='Afghanistan, Islamic Rep. of', ISO2='AF', ISO3='AFG', Indicator='Temperature change with respect to a baseline climatology, corresponding to the period 1951-1980', Unit='Degree Celsius', Source='Food and Agriculture Organization of the United Nations (FAO). 2022. FAOSTAT Climate Change, Climate Indicators, Temperature change. License: CC BY-NC-SA 3.0 IGO. Extracted from:\xa0https://www.fao.org/faostat/en/#data/ET. Accessed on 2023-03-28.', CTS_Code='ECCS', CTS_Name='Surface Temperature Change', CTS_Full_Descriptor='Environment, Climate Change, Climate and Weather, Surface Temperature Change', F1961=-0.126, F1962=-0.173, F1963=0.844, F1964=-0.751, F1965=-0.22, F1966=0.239, F1967=-0.348, F1968=-0.398, F1969=-0.513, F1970=0.843, F1971=0.642, F1972=-1.095, F1973=0.264, F1974=-0.466, F1975=-0.442, F1976=-0.291, F1977=0.544, F1978=0.12, F1979=0.38, F1980=0.655, F1981=0.558, F1982=-0.286, F1983=0.24, F1984=0.252, F1985=0.397, F1986=-0.015, F1987=0.493, F1988=1.035, F

In [12]:
df.select('Country','Indicator', 'ISO2', 'ISO3', 'F2019','F2020', 'F2021', 'F2022', 'F2023').show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+----+------+-----+-----+-----+-----+-----+
|             Country|           Indicator|ISO2|  ISO3|F2019|F2020|F2021|F2022|F2023|
+--------------------+--------------------+----+------+-----+-----+-----+-----+-----+
|Afghanistan, Isla...|Temperature chang...|  AF|   AFG|0.991|0.587|1.475|2.154|1.956|
|              Africa|Temperature chang...|null|AFRTMP|1.297|  1.2|1.396|0.996|1.485|
|             Albania|Temperature chang...|  AL|   ALB|1.751|1.567|1.589|1.585|2.122|
|             Algeria|Temperature chang...|  DZ|   DZA|1.107|1.905| 2.36|1.776|2.274|
|      American Samoa|Temperature chang...|  AS|   ASM|1.209|1.101|0.938|0.925|0.951|
+--------------------+--------------------+----+------+-----+-----+-----+-----+-----+
only showing top 5 rows

In [21]:
df.describe(['F2019','F2020', 'F2021', 'F2022', 'F2023']).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+------------------+------------------+------------------+------------------+
|summary|             F2019|             F2020|             F2021|             F2022|             F2023|
+-------+------------------+------------------+------------------+------------------+------------------+
|  count|               222|               220|               223|               223|               221|
|   mean| 1.429130630630631| 1.523422727272728|1.3098340807174886|  1.34895067264574|1.6473303167420823|
| stddev|0.4705686615270562|0.6246334438749978|0.5195694113857042|0.6784408943345038|0.6001559755099951|
|    min|             0.121|             0.059|            -0.569|            -0.698|             0.215|
|    max|             2.871|             3.669|             3.012|             3.512|             3.578|
+-------+------------------+------------------+------------------+------------------+------------------+

In [22]:
#filter country 
df['Country','Indicator', 'ISO2', 'ISO3', 'F2019','F2020', 'F2021', 'F2022', 'F2023'].filter(df['Country']=='Colombia').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+----+----+-----+-----+-----+-----+-----+
| Country|           Indicator|ISO2|ISO3|F2019|F2020|F2021|F2022|F2023|
+--------+--------------------+----+----+-----+-----+-----+-----+-----+
|Colombia|Temperature chang...|  CO| COL|1.278|1.338|0.824|0.671|1.265|
+--------+--------------------+----+----+-----+-----+-----+-----+-----+

In [17]:
#filter the records 
df.filter(df['Country']=='Colombia').select('Country','Indicator', 'ISO2', 'ISO3', 'F2019','F2020', 'F2021', 'F2022', 'F2023').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+----+----+-----+-----+-----+-----+-----+
| Country|           Indicator|ISO2|ISO3|F2019|F2020|F2021|F2022|F2023|
+--------+--------------------+----+----+-----+-----+-----+-----+-----+
|Colombia|Temperature chang...|  CO| COL|1.278|1.338|0.824|0.671|1.265|
+--------+--------------------+----+----+-----+-----+-----+-----+-----+

In [26]:
df['Country','Indicator', 'F2019','F2020', 'F2021', 'F2022', 'F2023'].filter(df['F2023'] >1.64).show()
#registros con diferencia en cambios de temperatura para el año 2023 mayor al promedio

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----+-----+-----+-----+-----+
|             Country|           Indicator|F2019|F2020|F2021|F2022|F2023|
+--------------------+--------------------+-----+-----+-----+-----+-----+
|Afghanistan, Isla...|Temperature chang...|0.991|0.587|1.475|2.154|1.956|
|             Albania|Temperature chang...|1.751|1.567|1.589|1.585|2.122|
|             Algeria|Temperature chang...|1.107|1.905| 2.36|1.776|2.274|
|            Americas|Temperature chang...|1.229|1.303|1.558|1.075|1.772|
|Andorra, Principa...|Temperature chang...|1.884| 2.48| 1.45|3.155|2.786|
|    Armenia, Rep. of|Temperature chang...|1.864|1.954|2.183|1.845|2.258|
|                Asia|Temperature chang...|1.388|1.506| 1.54|1.723|1.778|
|             Austria|Temperature chang...|2.375|2.322| 1.41|2.538|2.572|
| Azerbaijan, Rep. of|Temperature chang...|1.878|1.889|1.958|2.029| 2.16|
|        Bahamas, The|Temperature chang...|  1.6|1.715|0.714|1.304|1.688|
| Bahrain, Kingdom of|Temperature chan

In [27]:
#los diferentes paises
df.select('Country').distinct().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|             Country|
+--------------------+
|Iran, Islamic Rep...|
| Uzbekistan, Rep. of|
|              Guyana|
|          Czech Rep.|
| San Marino, Rep. of|
|        Saint Helena|
|Central African Rep.|
|           Argentina|
|              Angola|
|    Netherlands, The|
|             Albania|
|           Nicaragua|
|                Peru|
|             Somalia|
|          Martinique|
|            Suriname|
|              Norway|
|Ethiopia, The Fed...|
|          Guadeloupe|
|Micronesia, Feder...|
+--------------------+
only showing top 20 rows

In [28]:
#cantidad de paises distintos
df.select('Country').distinct().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

236

Los datos con los que contamos en la tabla de cambio de temperatura ya vienen agrupados, por lo que tenemos un solo indicador y todos los datos por lo que no tiene sentido usar la funcion df.groupBy()


In [40]:
from pyspark.sql.functions import udf
#Cambio de temperatura
def change_temp(F2023):
    try:
        F2023 = float(F2023) 
        if F2023 < 1.64:
            return 'Diferencia baja'
        else:
            return 'Diferencia alta'
    except ValueError:
        return 'Invalid value'  

change_udf = udf(change_temp, StringType())
df = df.withColumn('change_temp', cant_udf(df['F2023']))
df.select('change_temp').show(10, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+
|change_temp    |
+---------------+
|Diferencia alta|
|Diferencia baja|
|Diferencia alta|
|Diferencia alta|
|Diferencia baja|
|Diferencia alta|
|Diferencia alta|
|Diferencia baja|
|Diferencia baja|
|Diferencia baja|
+---------------+
only showing top 10 rows

ya con esta nueva columna podemos generar agrupaciones:

In [47]:
df_sin_null = df.filter(df["F2023"].isNotNull())
df_sin_null.groupBy('change_temp').agg({'F2023':'sum'}).show(5,False)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+------------------+
|change_temp    |sum(F2023)        |
+---------------+------------------+
|Diferencia alta|232.69            |
|Diferencia baja|131.36999999999998|
+---------------+------------------+

In [49]:
df_sin_null.groupBy('change_temp').agg({'F2023':'avg'}).show(5,False)
#promedio de la diferencia del cambio de temperatura del 2023 respecto al 2022

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+------------------+
|change_temp    |avg(F2023)        |
+---------------+------------------+
|Diferencia alta|2.154537037037037 |
|Diferencia baja|1.1625663716814156|
+---------------+------------------+

In [63]:

print(df_sin_null.groupBy('change_temp').agg({'F2023':'min'}).show(5,False))
print(df_sin_null.groupBy('change_temp').agg({'F2023':'max'}).show(5,False))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+----------+
|change_temp    |min(F2023)|
+---------------+----------+
|Diferencia alta|1.646     |
|Diferencia baja|0.215     |
+---------------+----------+

None
+---------------+----------+
|change_temp    |max(F2023)|
+---------------+----------+
|Diferencia alta|3.578     |
|Diferencia baja|1.639     |
+---------------+----------+

None

In [64]:
#Agregar la clasificación por medio de lambda
change_temp_udf = udf(lambda F2023: "Diferencia baja" if F2023 <= 1.646 else "Diferencia alta", StringType())

df.withColumn("change_temp2", change_temp_udf(df.F2023)).show(10,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+----------------------------+----+------+------------------------------------------------------------------------------------------------+--------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+--------------------------+----------------------------------------------------------------------------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-----+------+-----+------+------+------+------+------+------+------+------+------+------+------+-----+------+-----+-----+-----+-----+-----+-----+-----+-----+------+-----+-----+-----+-----+-----+-----+-----+-----+------+-----+-----+---------------+---------------+
|ObjectId|Country          

In [66]:
# eliminar duplicados
print(df_sin_null.count())
df_sin_null=df_sin_null.dropDuplicates()
print(df_sin_null.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

221
221

No cuenta con duplicados

In [71]:
df_new=df_sin_null.drop('CTF_Full_Descriptor')
df_new.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------------------+----+------+--------------------+--------------+--------------------+--------+--------------------+--------------------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+---------------+
|ObjectId|            Country|ISO2|  ISO3|           Indicator|          Unit|              Source|CTS_Code|            CTS_Name| CTS_Full_Descriptor| F1961| F1962| F1963| F1964| F1965| F1966| F1967| F1968| F1969| F1970| F1971| F1972| F1973| F1974| F1975| F1976| F1977| F1978| F1979| F1980| F1981| F1982| F1983| F1984| F1985| F1986| F1987| F1988| F1989| F1990| F1991| F1992| F1993| F1994| F1995| F1996|F1997|F1998|F1999

In [72]:
#target directory 
#pathcsv_out='../out/df_csv'
#pathcsv_out='gdrive/MyDrive/st1800-231/out/df_csv'
# hacia S3
pathcsv_out ='s3://trustedtrabajo1/trusted/spark_temp_change'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [73]:
df_new.coalesce(1).write.format("csv").option("header","true").save(pathcsv_out)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…